In [1]:
from nba_api.stats.endpoints import leaguegamelog, teamgamelog
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import time
import warnings

warnings.filterwarnings("ignore")

In [11]:
season = "2019-20"

In [12]:
teams_df = pd.DataFrame(teams.get_teams())
team_to_id = {team["full_name"]: team["id"] for team in teams_df.to_dict("records")}
team_to_id.update(
    {team["abbreviation"]: team["id"] for team in teams_df.to_dict("records")}
)
team_to_id.update({team["id"]: team["id"] for team in teams_df.to_dict("records")})
teams_df.head(5)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [13]:
# Get league game log for 2023 - 2024 season
game_log = leaguegamelog.LeagueGameLog(season=season).get_data_frames()
assert len(game_log) == 1
games_df = game_log[0]

# filter games_df to only include home games (prevent duplicating for both teams)
games_df = games_df[~games_df.MATCHUP.str.contains("@")]
print(games_df.shape, len(games_df.GAME_ID.unique()))

# add opponent team_id and team_abbreviation columns
games_df.loc[:, "OPP_TEAM_ABBREVIATION"] = games_df["MATCHUP"].str.split(" vs. ").str[1]
games_df.loc[:, "OPP_TEAM_ID"] = games_df["OPP_TEAM_ABBREVIATION"].map(team_to_id)

# adjust WL to 1 for win and 0 for loss
games_df.loc[:, "WL"] = games_df["WL"].map({"W": 1, "L": 0, 1: 1, 0: 0})

# get rid of useless columns
games_df = games_df[
    [
        "GAME_ID",
        "GAME_DATE",
        "TEAM_ID",
        "TEAM_ABBREVIATION",
        "OPP_TEAM_ID",
        "OPP_TEAM_ABBREVIATION",
        "PLUS_MINUS",
        "WL",
    ]
]

games_df.head(10)

(1059, 29) 1059


,GAME_ID,GAME_DATE,TEAM_ID,TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL
0,0021900001,2019-10-22,1610612761,TOR,1610612740,NOP,8,1
3,0021900002,2019-10-22,1610612746,LAC,1610612747,LAL,10,1
4,0021900005,2019-10-23,1610612753,ORL,1610612739,CLE,9,1
5,0021900007,2019-10-23,1610612748,MIA,1610612763,MEM,19,1
6,0021900008,2019-10-23,1610612755,PHI,1610612738,BOS,14,1
8,0021900012,2019-10-23,1610612756,PHX,1610612758,SAC,29,1
12,0021900004,2019-10-23,1610612754,IND,1610612765,DET,-9,0
14,0021900009,2019-10-23,1610612742,DAL,1610612764,WAS,8,1
16,0021900003,2019-10-23,1610612766,CHA,1610612741,CHI,1,1
17,0021900011,2019-10-23,1610612762,UTA,1610612760,OKC,5,1


In [14]:
def process_team_df(team_df):
    team_df["WL"] = team_df["WL"].map({"W": 1, "L": 0, 1: 1, 0: 0})
    team_df["P_W_PCT"] = team_df["W_PCT"].shift(-1)
    team_df["PAST_5"] = team_df["WL"].rolling(5).mean().shift(-5)
    team_df["PAST_5_PTS"] = team_df["PTS"].rolling(5).mean().shift(-5)
    team_df["W-L"] = team_df["W"] - team_df["L"]
    team_df["P_W-L"] = team_df["W-L"].shift(-1)
    team_df = team_df[
        [
            "Game_ID",
            "P_W_PCT",
            "PAST_5",
            "PAST_5_PTS",
            "P_W-L",
        ]
    ]
    return team_df

In [15]:
team_dfs = {}
for i, team in enumerate(teams_df["id"]):
    print(f"Getting game log for {teams_df['full_name'][i]}")
    team_log = teamgamelog.TeamGameLog(team_id=team, season=season).get_data_frames()
    assert len(team_log) == 1
    team_dfs[teams_df["abbreviation"][i]] = process_team_df(team_log[0])
    time.sleep(0.5)

Getting game log for Atlanta Hawks
Getting game log for Boston Celtics
Getting game log for Cleveland Cavaliers
Getting game log for New Orleans Pelicans
Getting game log for Chicago Bulls
Getting game log for Dallas Mavericks
Getting game log for Denver Nuggets
Getting game log for Golden State Warriors
Getting game log for Houston Rockets
Getting game log for Los Angeles Clippers
Getting game log for Los Angeles Lakers
Getting game log for Miami Heat
Getting game log for Milwaukee Bucks
Getting game log for Minnesota Timberwolves
Getting game log for Brooklyn Nets
Getting game log for New York Knicks
Getting game log for Orlando Magic
Getting game log for Indiana Pacers
Getting game log for Philadelphia 76ers
Getting game log for Phoenix Suns
Getting game log for Portland Trail Blazers
Getting game log for Sacramento Kings
Getting game log for San Antonio Spurs
Getting game log for Oklahoma City Thunder
Getting game log for Toronto Raptors
Getting game log for Utah Jazz
Getting game 

In [20]:
comp_team_dfs = pd.concat(
    [df.assign(TEAM_ABBREVIATION=abbr) for abbr, df in team_dfs.items()]
)

In [ ]:
df = games_df.merge(
    comp_team_dfs,
    left_on=["GAME_ID", "TEAM_ABBREVIATION"],
    right_on=["Game_ID", "TEAM_ABBREVIATION"],
    how="left",
)
df = df.merge(
    comp_team_dfs,
    left_on=["GAME_ID", "OPP_TEAM_ABBREVIATION"],
    right_on=["Game_ID", "TEAM_ABBREVIATION"],
    how="left",
    suffixes=("", "_OPP"),
)

TypeError: DataFrame.merge() got an unexpected keyword argument 'prefixes'

In [23]:
df.head(36)

,GAME_ID,GAME_DATE,TEAM_ID,TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL,Game_ID,P_W_PCT,PAST_5,PAST_5_PTS,P_W-L,Game_ID_OPP,P_W_PCT_OPP,PAST_5_OPP,PAST_5_PTS_OPP,P_W-L_OPP,TEAM_ABBREVIATION_OPP
0,0021900001,2019-10-22,1610612761,TOR,1610612740,NOP,8,1,0021900001,NaN,NaN,NaN,NaN,0021900001,NaN,NaN,NaN,NaN,NOP
1,0021900002,2019-10-22,1610612746,LAC,1610612747,LAL,10,1,0021900002,NaN,NaN,NaN,NaN,0021900002,NaN,NaN,NaN,NaN,LAL
2,0021900005,2019-10-23,1610612753,ORL,1610612739,CLE,9,1,0021900005,NaN,NaN,NaN,NaN,0021900005,NaN,NaN,NaN,NaN,CLE
3,0021900007,2019-10-23,1610612748,MIA,1610612763,MEM,19,1,0021900007,NaN,NaN,NaN,NaN,0021900007,NaN,NaN,NaN,NaN,MEM
4,0021900008,2019-10-23,1610612755,PHI,1610612738,BOS,14,1,0021900008,NaN,NaN,NaN,NaN,0021900008,NaN,NaN,NaN,NaN,BOS
5,0021900012,2019-10-23,1610612756,PHX,1610612758,SAC,29,1,0021900012,NaN,NaN,NaN,NaN,0021900012,NaN,NaN,NaN,NaN,SAC
6,0021900004,2019-10-23,1610612754,IND,1610612765,DET,-9,0,0021900004,NaN,NaN,NaN,NaN,0021900004,NaN,NaN,NaN,NaN,DET
7,0021900009,2019-10-23,1610612742,DAL,1610612764,WAS,8,1,0021900009,NaN,NaN,NaN,NaN,0021900009,NaN,NaN,NaN,NaN,WAS
8,0021900003,2019-10-23,1610612766,CHA,1610612741,CHI,1,1,0021900003,NaN,NaN,NaN,NaN,0021900003,NaN,NaN,NaN,NaN,CHI
9,0021900011,2019-10-23,1610612762,UTA,1610612760,OKC,5,1,0021900011,NaN,NaN,NaN,NaN,0021900011,NaN,NaN,NaN,NaN,OKC
